In [70]:
import pandas as pd
import os

In [71]:
def read_and_combine_files(ip_dir_path, gtype, op_dir_path):
    os.chdir(ip_dir_path)
    final_df = pd.DataFrame(columns=['NAME', 'B14001_002E', 'B14001_006E', 'B14001_007E', 'B14001_008E', 'B14001_009E', 'B14002_003E', 'B14002_013E', 'B14002_016E', 'B14002_019E', 'B14002_022E', 'B14002_027E', 'B14002_037E', 'B14002_040E', 'B14002_043E', 'B14002_046E', 'B15001_001E', 'B15001_002E', 'B15001_003E', 'B15001_011E', 'B15001_043E', 'B15001_044E', 'B15001_052E', 'B15002_001E', 'B15002_002E', 'B15002_019E', 'C15002A_001E', 'C15002A_002E', 'C15002A_007E', 'C15002B_001E', 'C15002B_002E', 'C15002B_007E', 'C15002H_001E', 'C15002H_002E', 'C15002H_007E', 'C15002I_001E', 'C15002I_002E', 'C15002I_007E', 'state', 'county', 'county subdivision', 'year'])
    for fl in os.listdir():
        if fl != '.DS_Store':
            fl_df = pd.read_csv(f'{ip_dir_path}/{fl}')
            year = fl.split('_')[4]
            js_year = year.split('.')[0]
            fl_df['year'] = js_year
            final_df = final_df.append(fl_df)
            
    final_df.to_csv(f'{op_dir_path}/educ_schl_10_20_{gtype}.csv', index=False)

In [72]:
## county_subd files
read_and_combine_files(ip_dir_path = '/Users/salma/Research/dissertation_work/data/educ_schl_acs/c_subd',
                      gtype = 'csubd',
                      op_dir_path = '/Users/salma/Research/dissertation_work/data/educ_schl_acs/all')

In [73]:
## county files
read_and_combine_files(ip_dir_path = '/Users/salma/Research/dissertation_work/data/educ_schl_acs/cnty',
                      gtype = 'county',
                      op_dir_path = '/Users/salma/Research/dissertation_work/data/educ_schl_acs/all')

In [74]:
## place files
read_and_combine_files(ip_dir_path = '/Users/salma/Research/dissertation_work/data/educ_schl_acs/place',
                      gtype = 'place',
                      op_dir_path = '/Users/salma/Research/dissertation_work/data/educ_schl_acs/all')

### Create FIPS columns for county and csubd files

In [78]:
# utility function to update fips column values as per the required length
def update_fips_code_len(fips_code, fp_type):
    
    # placeholder to assign required code length based on whether it is a city, county or state fips code.
    # For now cnty and placefips ar of reqd len coz considered as strings. but can have below code for future use
    
    req_code_len = 0  

    fp_code_len = fips_code.__len__()

    if fp_type == 'city':
        req_code_len = 5
    elif fp_type == 'county':
        req_code_len = 3
    elif fp_type == 'state':
        req_code_len = 2

    if fp_code_len < req_code_len:
        while fips_code.__len__() < req_code_len:
            fips_code = '0'*(req_code_len-fp_code_len) + fips_code
            return fips_code
    else:
        return fips_code


#### County file

In [89]:
cnty_df = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/all/educ_schl_10_20_county.csv')

## county subdivision column in place fips. For now, we wil just create place fips and place under it.
## Later after everything is appended, we will rename at once

cnty_df['county'] = cnty_df['county'].astype(str).apply(update_fips_code_len, args=('county',))
cnty_df['county subdivision'] = '99' + cnty_df['county']
cnty_df.rename({'county subdivision': 'place_fips'}, axis=1, inplace=True)
print(cnty_df.shape[0])

34589


### Append all three

In [90]:
## Read csubd and place ##
csubd_df = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/all/educ_schl_10_20_csubd.csv')
csubd_df.rename({'county subdivision': 'place_fips'}, axis=1, inplace=True)
print(csubd_df.shape[0])

place_df = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/all/educ_schl_10_20_place.csv')
place_df.drop(['county subdivision'], axis=1, inplace=True)
place_df.rename({'place': 'place_fips'}, axis=1, inplace=True)
print(place_df.shape[0])

405263
321995


In [92]:
## Append ##
educ_schl_acs_10_20 = pd.concat([cnty_df, csubd_df, place_df], sort=False, ignore_index=True)
print(educ_schl_acs_10_20.shape[0])

761847


In [93]:
### Rename ###
educ_schl_acs_10_20.rename({'B14001_002E':'schl_enrl_tot',
                            'B14001_006E':'schl_enrl_grades_5_8',
                            'B14001_007E':'schl_enrl_grades_9_12',
                            'B14001_008E':'schl_enrl_ugrad',
                            'B14001_009E':'schl_enrl_grad',
                            'B14002_003E':'schl_enrl_m',
                            'B14002_013E':'schl_enrl_grades_m_5_8',
                            'B14002_016E':'schl_enrl_grades_m_9_12',
                            'B14002_019E':'schl_enrl_m_ugrad',
                            'B14002_022E':'schl_enrl_m_grad',
                            'B14002_027E':'schl_enrl_f',
                            'B14002_037E':'schl_enrl_grades_f_5_8',
                            'B14002_040E':'schl_enrl_grades_f_9_12',
                            'B14002_043E':'schl_enrl_f_ugrad',
                            'B14002_046E':'schl_enrl_f_grad',
                            'B15001_001E':'educ_atnmnt_18ao',
                            'B15001_002E':'educ_atnmnt_18ao_m',
                            'B15001_003E':'educ_atnmnt_18_24_m',
                            'B15001_011E':'educ_atnmnt_25_34_m',
                            'B15001_043E':'educ_atnmnt_18ao_f',
                            'B15001_044E':'educ_atnmnt_18_24_f',
                            'B15001_052E':'educ_atnmnt_25_34_f',
                            'B15002_001E':'educ_atnmnt_25ao',
                            'B15002_002E':'educ_atnmnt_25ao_m',
                            'B15002_019E':'educ_atnmnt_25ao_f',
                            'C15002B_001E':'educ_atnmnt_25ao_b',
                            'C15002B_002E':'educ_atnmnt_25ao_b',
                            'C15002B_007E':'educ_atnmnt_25ao_b',
                            'C15002H_001E':'educ_atnmnt_25ao_w',
                            'C15002H_002E':'educ_atnmnt25ao_m_w',
                            'C15002H_007E':'educ_atnmnt_25ao_f_w',
                            'C15002I_001E':'educ_atnmnt_25ao_h',
                            'C15002I_002E':'educ_atnmnt_25ao_h',
                            'C15002I_007E':'educ_atnmnt_25ao_h',
                            'state': 'STATEFP',
                            'county': 'CNTY'}, axis=1, inplace=True)

educ_schl_acs_10_20.to_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/educ_schl_acs_10_20.csv', index=False)

In [95]:
educ_schl_acs_10_20.isnull().sum()

NAME                            0
schl_enrl_tot                   0
schl_enrl_grades_5_8            0
schl_enrl_grades_9_12           0
schl_enrl_ugrad                 0
schl_enrl_grad                  0
schl_enrl_m                     0
schl_enrl_grades_m_5_8          0
schl_enrl_grades_m_9_12         0
schl_enrl_m_ugrad               0
schl_enrl_m_grad                0
schl_enrl_f                     0
schl_enrl_grades_f_5_8          0
schl_enrl_grades_f_9_12         0
schl_enrl_f_ugrad               0
schl_enrl_f_grad                0
educ_atnmnt_18ao                0
educ_atnmnt_18ao_m              0
educ_atnmnt_18_24_m             0
educ_atnmnt_25_34_m             0
educ_atnmnt_18ao_f              0
educ_atnmnt_18_24_f             0
educ_atnmnt_25_34_f             0
educ_atnmnt_25ao                0
educ_atnmnt_25ao_m              0
educ_atnmnt_25ao_f              0
C15002A_001E                    0
C15002A_002E                    0
C15002A_007E                    0
educ_atnmnt_25